# Algoritmo genético

In [498]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import pandas as pd
import random as rnd
from random import random, uniform
import functools

# Métodos de Codificación 

## Codificación Binaria

In [499]:
def int2bin(entero,param={"numbits":10}):
  #print(param["numbits"])
  code="{0:0"+str(param["numbits"])+"b}"
  Xbin=code.format(int(round(entero)))
  return Xbin

def bin2int(Binario,param=False):
  entero=int(Binario, 2)
  return entero

In [500]:
Binario=int2bin(6,param={"numbits":10})
print(Binario)
bin2int(Binario)

0000000110


6

## Codificación Binaria para números reales

In [501]:
def BinaryPrecision(Xu,Xl,e):
  n=math.log2((Xu-Xl)/e)
  return int(n)

def Binary2Real(Xu,Xl,n,Xbin):
  Xbin=int(Xbin, 2)
  Value=Xl+((Xu-Xl)/((2**n)-1))*Xbin
  return Value

def Real2Binary(Xu,Xl,n,Value):
  Xbin=(Value-Xl)/((Xu-Xl)/((2**n)-1))
  code="{0:0"+str(n)+"b}"
  Xbin=code.format(int(round(Xbin)))
  return Xbin

def codeBinary2Real(Xbin, param={"Xu":0,"Xl":0,"n":0}):
  Xbin=int(Xbin, 2)
  Value=param["Xl"]+((param["Xu"]-param["Xl"])/((2**param["n"])-1))*Xbin
  return [Value]

def codepoblacionReal2bin(poblacion,limits):
  code=list()
  preV=list()
  for parameter in limits:
    preV.append(BinaryPrecision(parameter[0],parameter[1],parameter[2]))
  for individuo in poblacion:
    indcode=""
    for index,parameter in enumerate(individuo):
      indcode=indcode+Real2Binary(limits[index][0],limits[index][1],preV[index],parameter)
    code.append(indcode)
  return code,preV

In [502]:
def decodificador(population,code,codeParam=False):  
  aptitud=list()
  for individuo in population:
    aptitud.append(code(individuo,codeParam))
    
  return aptitud

In [503]:
n=BinaryPrecision(10,2,.01)
print(n)
Nbin=Real2Binary(10,2,n,8)
print(Nbin)
Binary2Real(10,2,n,Nbin)

9
101111111


7.996086105675147

In [504]:
def noCodification(XbinIN, param=False):
  return XbinIN

## Cálculo de la aptitud  de una población 

In [505]:
def AptitudProporcional(objective,population,code,codeParam=False,case="minimo"):   
  aptitud=list()
  for individuo in population:
    if case == "minimo":
      aptitud.append(1/objective(code(individuo,codeParam)))
    elif case == "maximo":
      aptitud.append(objective(code(individuo,codeParam)))
    else:
      aptitud.append(1/objective(code(individuo,codeParam)))
  fS=sum(aptitud)
  prob=list()
  for app in aptitud:
    prob.append(float(app/fS))
  return aptitud,prob

Ejemplo:  Problema de la parábola 

In [506]:
def prob_parabola(Parametros):
  #definición de los objetivos
  x=Symbol('x')
  A=x**2
  objetivo=A.subs([[x,Parametros[0]]])
  
  return objetivo

In [507]:
populus=[[6],[20],[10],[12]]
limits=[[25,0,.5]]
binpopulus,n=codepoblacionReal2bin(populus,limits)
out=AptitudProporcional(prob_parabola,binpopulus,codeBinary2Real,codeParam={"Xu":25,"Xl":0,"n":5},case="minimo")
out=AptitudProporcional(prob_parabola,binpopulus,codeBinary2Real,codeParam={"Xu":25,"Xl":0,"n":5},case="maximo")

# Técnicas de selección



## Stochastic Universal Sampling (SUS)

In [508]:
def SUS(Poblacion,Probabilidades):
    #Total Fitness pop
    N=2
    padres=list()
    for i in range(int(len(Probabilidades)/N)):
        F=len(Probabilidades)
        P=(F/N)/F
        start=uniform(0, P)    
        points= [start+ i*P for i in range(N)]    
        for j in points:
            index=0
            suma=Probabilidades[index]        
            while suma<=j:
                index+=1
                suma+=Probabilidades[index]                    
            padres.append(Poblacion[index])        
    return(padres)  


In [509]:
 SobranteEstocastico(binpopulus,out[1])

['01111', '11001', '11001', '01100']

# Técnicas de cruza

## Cruza en un punto

In [510]:
def cruza1Punto(padres,param=False):
  hijos=list()
  i=0
  while i<len(padres):
    pareja=copy.deepcopy(padres[i:i+2])
    cruze=rnd.randint(1,len(padres[i])-1)
    hijo1=pareja[0][:cruze]+pareja[1][cruze:]
    hijo2=pareja[1][:cruze]+pareja[0][cruze:]
    i=i+2
    hijos.append(hijo1)
    hijos.append(hijo2)
  return hijos

## Order crossover 

In [511]:
def orderCrossover(padres,param=False): 
  hijos=list()
  i=0
  while i<len(padres):
    pareja=copy.deepcopy(padres[i:i+2])
    i=i+2
    hijo1=[None] * len(padres[0])
    hijo2=[None] * len(padres[0])
    selec=rnd.randint(0,len(padres[0]))
    selec2=rnd.randint(0,len(padres[0]))
    if selec<selec2:
      start=selec
      end=selec2
    else:
      start=selec2
      end=selec
    hijo1[start:end]=pareja[0][start:end]
    hijo2[start:end]=pareja[1][start:end]
    for elemento in pareja[0]:
      if elemento not in hijo2:
        hijo2[hijo2.index(None)]=elemento

    for elemento in pareja[1]:
      if elemento not in hijo1:
        hijo1[hijo1.index(None)]=elemento


    hijos.append(hijo1)
    hijos.append(hijo2)
  return hijos

## Mutación

### Mutación Binaria 

In [512]:
def mutacionBinaria(individuos,parameter={"umbral":0.1}):
  hijos=copy.deepcopy(individuos)
  for elem,individuo in enumerate(individuos):
    for index,gen in enumerate(individuo):
      if random()<parameter["umbral"]:
        mutacion=str(abs(int(gen)-1))
        hijos[elem]=individuo[:index]+mutacion+individuo[index+1:]
  return hijos

## Algoritmo Genético Completo 

In [513]:
def AGenetico(problema,poblacion,code,NIteracion,Aptitud=AptitudProporcional,
              Seleccion=SUS,cruza=cruza1Punto,
              mutacion=mutacionBinaria,
              SelecParameter=False ,cruzaParameter=False,mutacionParameters=False,
              codeParameter=False,case="minimo"):
  Historico=list()
  for i in range(NIteracion):
    aptitud,probabilidad=Aptitud(problema,poblacion,code,codeParameter,case)
    Historico.append(aptitud)
    padres=Seleccion(poblacion,probabilidad)
    hijos=cruza(padres,cruzaParameter)
    hijos=mutacion(hijos,mutacionParameters)
    poblacion=copy.deepcopy(hijos)
  decodePop=decodificador(poblacion,code,codeParameter)
  return poblacion,decodePop,Historico
  
  
  

## Problemas con números reales 

In [514]:
def prob_caja_volumen_restringido(Parametros):
  '''Este modelo calcula el área de un prisma cuadrado con un volumen fijo 
  de 216 milímetros '''
  #definición de los objetivos
  x=Symbol('x')
  A=(864/x)+x**2
  objetivo=A.subs([[x,Parametros[0]]])
  
  return objetivo

In [515]:
def prob_elipse(Parametros):
  ''' modelo de la elipse'''
  #definición de los objetivos
  x=Symbol('x')
  y=Symbol('y')

  A=x**2+y**2
  objetivo=A.subs([[x,Parametros[0]],[y,Parametros[1]]])
  
  return objetivo

In [516]:
def prob_parabola(Parametros):
  #definición de los objetivos
  x=Symbol('x')
  A=x**2
  objetivo=A.subs([[x,Parametros[0]]])
  
  return objetivo

In [517]:
Dist = pd.DataFrame([[0,1,2,3,1,2,3,4],[1,0,1,2,2,1,2,3],[2,1,0,1,3,2,1,2],
                      [3,2,1,0,4,3,2,1],[1,2,3,4,0,1,2,3],[2,1,2,3,1,0,1,2],
                      [3,2,1,2,2,1,0,1],[4,3,2,1,3,2,1,0]],
                    columns=["A","B","C","D","E","F","G","H"],
                    index=["A","B","C","D","E","F","G","H"])
  
Flow = pd.DataFrame([[0,5,2,4,1,0,0,6],[5,0,3,0,2,2,2,0],[2,3,0,0,0,0,0,5],
                      [4,0,0,0,5,2,2,10],[1,2,0,5,0,10,0,0],[0,2,0,2,10,0,5,1],
                      [0,2,0,2,0,5,0,10],[6,0,5,10,0,1,10,0]],
                    columns=["A","B","C","D","E","F","G","H"],
                    index=["A","B","C","D","E","F","G","H"])

def QAP(order):
  #Definición de las matrices 
  #Matriz de distancia 
  order=[order]
  New_Dist_DF = Dist.reindex(columns=order[0], index=order[0])
  New_Dist_Arr = np.array(New_Dist_DF)
  Objfun1_start = pd.DataFrame(New_Dist_Arr*Flow)
  Objfun1_start_Arr = np.array(Objfun1_start)


  sum_start = sum(sum(Objfun1_start_Arr))
  
  return sum_start

In [518]:
QAP(["B","D","A","E","C","F","G","H"])

336

In [519]:
# ejemplo problema de volumen restringido

poblacion=[[0.5],[1.0],[1.5],[2],[2.5],[3.0],[3.5],[4.0],[4.5],[5],[5.5],[6.0],[6.5],[7.0],[8.0],[8.5]]
limits=[[10,0.5,0.2]]
binpopulus,n=codepoblacionReal2bin(poblacion,limits)
Poblacion,decodePop,Hist=AGenetico(prob_caja_volumen_restringido,binpopulus,codeBinary2Real,10,
          Aptitud=AptitudProporcional,Seleccion=SUS,
          cruza=cruza1Punto,mutacion=mutacionBinaria,codeParameter={"Xu":10,"Xl":0.5,"n":n[0]},
          SelecParameter=False, cruzaParameter=False,mutacionParameters={"umbral":0.01},
          case="minimo")
print("Poblacion final:  ")
print(Poblacion)
print(decodePop)


Poblacion final:  
['10111', '11010', '10111', '11010', '10111', '11010', '11110', '10101', '10101', '10110', '10100', '11010', '11110', '10101', '11110', '10100']
[[7.548387096774194], [8.467741935483872], [7.548387096774194], [8.467741935483872], [7.548387096774194], [8.467741935483872], [9.693548387096774], [6.935483870967742], [6.935483870967742], [7.241935483870968], [6.629032258064516], [8.467741935483872], [9.693548387096774], [6.935483870967742], [9.693548387096774], [6.629032258064516]]


In [520]:
#QAP
poblacion=[["B","D","A","E","C","F","G","H"],["A","B","D","H","F","C","G","E"],["E","C","F","B","D","A","G","H"],["C","G","A","B","D","H","F","E"],["E","C","G","H","F","B","D","A"],["C","G","H","F","E","A","B","D"]]
Poblacion,decodePop,Hist=AGenetico(QAP,poblacion,noCodification,10,
          Aptitud=AptitudProporcional,Seleccion=SUS,
          cruza=orderCrossover,mutacion=mutacionInsercion,codeParameter=False,
          SelecParameter=False, cruzaParameter=False,mutacionParameters={"umbral":0.01},
          case="minimo")
print("Poblacion final:  ")
print(Poblacion)
print(decodePop)


Poblacion final:  
[['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'C', 'D', 'F', 'H', 'G'], ['B', 'A', 'E', 'C', 'D', 'F', 'H', 'G'], ['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'F', 'D', 'C', 'G', 'H']]
[['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'C', 'D', 'F', 'H', 'G'], ['B', 'A', 'E', 'C', 'D', 'F', 'H', 'G'], ['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'C', 'D', 'F', 'G', 'H'], ['B', 'A', 'E', 'F', 'D', 'C', 'G', 'H']]
